# Extração de informações relevantes dos Extratos das Sessões de Julgamento dos PAS da CVM entre 2000 e 2023.

## Objetivo do código
O objetivo do código em questão é extrair, dentro do possível, permitido computacionalmente, informações sobre as decisões dos PAS da CVM, entre 2000 e 2023. 

Os documentos usados foram os Extratos das Sessões de Julgamento dos PAS, disponíveis para download no site oficial da CVM. Os Extratos das Sessões de Julgamento são os documentos publicados na sessão de julgamento, que acontece após a apresentação de defesa por parte dos acusados. Neles, ficam registrados os votos de cada um dos diretores presentes, além de outras informações relevantes. Para mais informações, consultar o codebook disponível no Github (https://github.com/tomasrribeiro/corporate-fraud-in-brazil).
    
O objetivo é colher 5 informações para cada documento. São elas:
1. Identificador do processo (*ID*): cada processo tem a ele um código associado. É importantíssimo que tenhamos esse código registrado para poder indexar quando necessário. O número do processo é colhido usando hífens no lugar de barras e pontos, para facilitar a indexação. Exemplo para um processo aleatório: "19957-004381-2021-68".
2. Data do julgamento (*Data*): a data em que a Sessão de Julgmento ocorre. Todo processo têm registrado em algum lugar a data de ocorrência. Ela pode estar em alguns formatos --- separada por barras ou por extenso. Exemplos: "12/12/2004" e "12 de dezembro de 2004"
3. Lista de acusados (*Acusados*): cada processo têm a ele associado uma lista de acusados, que pode conter um ou mais nomes, sendo eles pessoas físicas ou jurídicas. O objetivo é colher, para cada documento, a lista de acusado associada. Armazenamos a inforação no formato de uma lista dentro da base de dados. Exemplo colhido de um processo: "[Letícia Ferreira Duarte do Valle, Le Valle Agente Autônomo de Investimentos S/S Ltda.]".
4. Assunto ou ementa (*Assunto*): no cabeçalho de cada processo, temos uma breve descrição dos fatos. Normalmente, tem de 1 a 4 parágrafos e é importante pois dá uma visão geral do que está sendo julgado no processo. Exemplo de um processo: "Emissão de Parecer sem Abstenção de Opinião, em descumprimento ao item 21 da NBC T11-IT5, aprovada pela Resolução CFC Nº 830/98, que, por seu turno, representa uma infração ao art. 20 da Instrução CVM nº 308/99. Multas."
5. Decisão (*Decisão*): no cabeçalho, normalmente depois do assunto, temos uma breve descrição da decisão do Colegiado na Sessão de Julgamento. Exemplo, estraído de um processo: "Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, por maioria de votos, decidiu aplicar aos acusados BC Control Auditores Independentes S/S e Sandro Casagrande a penalidade de multa pecuniária individual no valor de R$ 75.000,00, pelo descumprimento do item 21 da NBC T11-IT 5, que representa infração ao art. 20 da Instrução CVM nº 308/99."

## Método
Usamos o pacote *pdfplumber* para colher as informações. Além disso, usamos expressões regulares para delimitar cada um dos tópicos do documento. Essas serão exploradas mais a fundo.

A análise foi feita com base no texto das duas primeiras páginas. Apesar desse limite não ser oficial, para todos os processo que observamos, o "cabeçalho" (parte que contém essas 5 informações ou a maioria delas) estava contido nesse limite.

O código foi construído com base em uma amostra escolhida aleatoriamente (n = 20, seed = 42) dentre os processos. A partir dessa amostra e da análise dos documentos nela presentes, as expressões regulares e a ordem e os métodos de colheita de informações foram adaptados, de forma a ter a maior eficiência possível, dentro da amostra. Se ela for represnetativa (acreditamos que, atéc erto ponto, depois de inúmeras leituras "na diagonal", ela seja), esperamos conseguir resultados semelhantes ao generalizar o código para todos os arquivos entre 2000 e 2023. Esse código também está disponível no Github.

Agora, vamos analisar os critérios utilizados, por variável:

### *ID*:
Para coletar o ID dos processos, usamos o nome (do arquivo) de cada um dos pdfs. Ao baixar a pasta com os arquivos dos Extratos das Sessões de Julgamento, notamos que os nomes estavam muito bem padronizados, para cada um dos documentos. Portanto, o que o código faz, para cada um dos documentos, é coletar o nome do arquivo e adicioná-lo à coluna de *ID*.

Na amostra aleatória que colhemos, tivemos 100% dos identificadores de processo colhidos, dos quais 100% estavam corretos.

### *Data*:
Para colher as datas do julgamento, usamos expressões regulares que foram construídas com base na observação da amostra. 

Para processos mais recentes, é fácil notar que normalmente, as primeiras páginas se iniciam com a expressão "Data do julgamento: XX/XX/20XX". Para todos os processos que tinham essa expressão regular, extraímos exatamente essa expressão.

Já para os processos mais antigos, normalmente a parte preliminar dos documentos termina com a data, por extenso, da seguinte forma: "Rio de Janiero, XX de mês (janeiro, fevereiro, etc.) de 20XX." Usamos essa expressão regular, quando contida nas duas primeira páginas, para extrair a data da Sessão de Julgamento daquele PAS. 

Sobre a eficácia: para a amostra de 20 aruquivos, colhemos a data em 18 (90%) e ela estava correta em todas as ocasiões em que foi colhida. Para os outros dois arquivos, não conseguimos colher a data por que ela não estava disponível nas duas primeiras páginas do documento (o documento estava desconfigurado). Em um dos documentos, por exemplo, a data estava na página 10. Isso exigiria uma capacidade computacional e um tempo de processamento que é desnecessariamente grande (é melhor extrair as informações faltantes manualmente).

### *Acusados*:
Novamente, usamos expressões regulares com base na observação da amostra. Na maior parte dos arquivos, notamos que a lista de acusados vem no formato "Acusados/Interessados/Indiciados: XXXX", seguida do assunto/ementa. Configuramos as expressões regulares com base nisso.

Sobre a eficácia: para a amostra de 20 arquivos, coletamos a lista de acusados em 20 (100%) e ela estava correta para 19 dos 20 arquivos (95%). Até mesmo os arquivos desfigurados, mencionados na sessão anterior, tinham uma lista de acusados nas primeiras páginas. Por isso, imaginando uma amostra representativa, esperamos ter boa acurácia nesse tópico. 

A lista de acusados colhida de maneira errônea foi graças a uma vírgula no meio do nome de uma empresa, de forma qu o seu nome ("New Way Técnicas Administrativas, Assessoria e Planejamento Ltda."), ficasse separado em duas strings, pela vírgula. Acreditamos que a probabilidade de ocorrência desse tipo de erro seja baixa para a maior parte dos processos, mas é um tópico de atenção.

### *Assunto*:
Para captar o assunto dos processos, usamos expressões regulares. Notamos que normalmente, o assunto segue um formato padrão: "Assunto/Ementa: XXX", eestá seguido da palavra "Decisão". 

Para o teste com a amostra de 20 arquivos, captamos o assunto de 18 (90%), dentre os quais 18 estavam corretos (100%). Os arquivos com os quais tivemos problemas foram os mesmos da seção de datas, graças à desfiguração. Ela vai ser discutida na seção posterior.

### *Decisão*:
Normalmente, a decisão é introduzida sempre pela frase "Vistos, relatados e discutidos os autos". Usamos essa expressão regular para iniciar a captura. Ela estava presente em todas as decisões que captamos.

Além disso, normalmente termina com uma expressão específica. Normalmente, depois de relatar a decisão, o relator dá o prazo para apresentação de recursos (30 dias, em todos os arquivos que obervamos). Por isso, usamos a expressão "30 dias" e suas variações como uma das "stopwords". Outra expressão muito usada é a data da sessão de julgamento, no formato por extenso. Usamos ela também. Além disso, em alguns casos, especialmente nos documentos mais recentes que analisamos, temos a expressão "Participaram desta Sessão", ou o fim da segunda página, como o fim da parte inicial dos autos.

Para o teste com a amostra de 20 arquivos, captamos a decisão em 18 (90%), dentre as quais todas continham as partes que julgamos relevantes (100%). Novamente, o problema foi na captação da decisão dos mesmos dois arquivos desfigurados. Apesar disso, houve, em muitos casos, a captura de mais texto do que necessário (uma frase, ou alguns parágrafos a mais). Isso é um ponto de atenção, mas em todos os casos a decisão foi captada por completo (acreditamos que o limite de duas p´ginas na análise dos documentos foi importante nesse sentido.

## Possíveis perdas de dados (desconfiguração, documentos escaneados, etc.)
Como comentado na seção anterior, observamos, para dois documentos, problemas na colheita dos dados. Em ambos os casos, os documentos não tinham a primeira página na estrutura que esperávamos (Acusados, Assunto, Decisão). Muito pelo contrário. O que aconteceu foi que não conseguimos colher esses dados por que eles não existiam dentro dos arquivos. Apesar disso, os dados disponíveis foram completamente colhidos. Isso, acreditamos é um indicativo de sucesso na acurária do código (nenhuma informação falsa/incorreta colhida, nenhum "falso positivo").

No entanto, é importante saber, obviamente, que o código em questão não será capaz de colher a totalidade das informações dentro da pasta de mais de 900 arquivos baixados do site da CVM, mas acreditamos que boa parte estará lá.

Outros problema que certamente vamos enfrentar: uma pequena parte (só nos deparamos com um até o momento) dos processos são imagens escaneadas dos autos dos processos. Nesse caso, o código não vai funcionar e as informações terão de ser registradas manualmente.

# Extração de informações relevantes dos Extratos das Sessões de Julgamento dos PAS da CVM entre 2000 e 2023.

## Objetivo do código
O objetivo do código em questão é extrair, dentro do possível permitido computacionalmente, informações sobre as decisões dos PAS da CVM, entre 2000 e 2023.

Os documentos usados foram os Extratos das Sessões de Julgamento dos PAS, disponíveis para download no site oficial da CVM. Os Extratos das Sessões de Julgamento são os documentos publicados na sessão de julgamento, que acontece após a apresentação de defesa por parte dos acusados. Neles, ficam registrados os votos de cada um dos diretores presentes, além de outras informações relevantes. Para mais informações, consultar o codebook disponível no Github (https://github.com/tomasrribeiro/corporate-fraud-in-brazil).

O objetivo é colher 5 informações para cada documento. São elas:
1. Identificador do processo (*ID*): cada processo tem a ele um código associado. É importantíssimo que tenhamos esse código registrado para poder indexar quando necessário. O número do processo é colhido usando hífens no lugar de barras e pontos, para facilitar a indexação. Exemplo para um processo aleatório: "19957-004381-2021-68".
2. Data do julgamento (*Data*): a data em que a Sessão de Julgamento ocorre. Todo processo tem registrado em algum lugar a data de ocorrência. Ela pode estar em alguns formatos --- separada por barras ou por extenso. Exemplos: "12/12/2004" e "12 de dezembro de 2004".
3. Lista de acusados (*Acusados*): cada processo tem a ele associado uma lista de acusados, que pode conter um ou mais nomes, sendo eles pessoas físicas ou jurídicas. O objetivo é colher, para cada documento, a lista de acusados associada. Armazenamos a informação no formato de uma lista dentro da base de dados. Exemplo colhido de um processo: "[Letícia Ferreira Duarte do Valle, Le Valle Agente Autônomo de Investimentos S/S Ltda.]".
4. Assunto ou ementa (*Assunto*): no cabeçalho de cada processo, temos uma breve descrição dos fatos. Normalmente, tem de 1 a 4 parágrafos e é importante, pois dá uma visão geral do que está sendo julgado no processo. Exemplo de um processo: "Emissão de Parecer sem Abstenção de Opinião, em descumprimento ao item 21 da NBC T11-IT5, aprovada pela Resolução CFC Nº 830/98, que, por seu turno, representa uma infração ao art. 20 da Instrução CVM nº 308/99. Multas."
5. Decisão (*Decisão*): no cabeçalho, normalmente depois do assunto, temos uma breve descrição da decisão do Colegiado na Sessão de Julgamento. Exemplo, extraído de um processo: "Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, por maioria de votos, decidiu aplicar aos acusados BC Control Auditores Independentes S/S e Sandro Casagrande a penalidade de multa pecuniária individual no valor de R$ 75.000,00, pelo descumprimento do item 21 da NBC T11-IT 5, que representa infração ao art. 20 da Instrução CVM nº 308/99."

## Método
Usamos o pacote *pdfplumber* para colher as informações. Além disso, usamos expressões regulares para delimitar cada um dos tópicos do documento. Essas serão exploradas mais a fundo.

A análise foi feita com base no texto das duas primeiras páginas. Apesar desse limite não ser oficial, para todos os processos que observamos, o "cabeçalho" (parte que contém essas 5 informações ou a maioria delas) estava contido nesse limite.

O código foi construído com base em uma amostra escolhida aleatoriamente (n = 20, seed = 42) dentre os processos. A partir dessa amostra e da análise dos documentos nela presentes, as expressões regulares e a ordem e os métodos de colheita de informações foram adaptados, de forma a ter a maior eficiência possível, dentro da amostra. Se ela for representativa (acreditamos que, até certo ponto, depois de inúmeras leituras "na diagonal", ela seja), esperamos conseguir resultados semelhantes ao generalizar o código para todos os arquivos entre 2000 e 2023. Esse código também está disponível no Github.

Agora, vamos analisar os critérios utilizados, por variável:

### *ID*:
Para coletar o ID dos processos, usamos o nome (do arquivo) de cada um dos PDFs. Ao baixar a pasta com os arquivos dos Extratos das Sessões de Julgamento, notamos que os nomes estavam muito bem padronizados, para cada um dos documentos. Portanto, o que o código faz, para cada um dos documentos, é coletar o nome do arquivo e adicioná-lo à coluna de *ID*.

Na amostra aleatória que colhemos, tivemos 100% dos identificadores de processo colhidos, dos quais 100% estavam corretos.

### *Data*:
Para colher as datas do julgamento, usamos expressões regulares que foram construídas com base na observação da amostra.

Para processos mais recentes, é fácil notar que, normalmente, as primeiras páginas se iniciam com a expressão "Data do julgamento: XX/XX/20XX". Para todos os processos que tinham essa expressão regular, extraímos exatamente essa expressão.

Já para os processos mais antigos, normalmente a parte preliminar dos documentos termina com a data, por extenso, da seguinte forma: "Rio de Janeiro, XX de mês (janeiro, fevereiro, etc.) de 20XX." Usamos essa expressão regular, quando contida nas duas primeiras páginas, para extrair a data da Sessão de Julgamento daquele PAS.

Sobre a eficácia: para a amostra de 20 arquivos, colhemos a data em 18 (90%) e ela estava correta em todas as ocasiões em que foi colhida. Para os outros dois arquivos, não conseguimos colher a data porque ela não estava disponível nas duas primeiras páginas do documento (o documento estava desconfigurado). Em um dos documentos, por exemplo, a data estava na página 10. Isso exigiria uma capacidade computacional e um tempo de processamento que é desnecessariamente grande (é melhor extrair as informações faltantes manualmente).

### *Acusados*:
Novamente, usamos expressões regulares com base na observação da amostra. Na maior parte dos arquivos, notamos que a lista de acusados vem no formato "Acusados/Interessados/Indiciados: XXXX", seguida do assunto/ementa. Configuramos as expressões regulares com base nisso.

Sobre a eficácia: para a amostra de 20 arquivos, coletamos a lista de acusados em 20 (100%) e ela estava correta para 20 dos 20 arquivos (100%). Até mesmo os arquivos desfigurados, mencionados na seção anterior, tinham uma lista de acusados nas primeiras páginas. Por isso, imaginando uma amostra representativa, esperamos ter boa acurácia nesse tópico.

A lista de acusados colhida de maneira errônea foi graças a uma vírgula no meio do nome de uma empresa, de forma que o seu nome ("New Way Técnicas Administrativas, Assessoria e Planejamento Ltda.") ficasse separado em duas strings, pela vírgula. Acreditamos que a probabilidade de ocorrência desse tipo de erro seja baixa para a maior parte dos processos, mas é um tópico de atenção.

### *Assunto*:
Para captar o assunto dos processos, usamos expressões regulares. Notamos que, normalmente, o assunto segue um formato padrão: "Assunto/Ementa: XXX", e está seguido da palavra "Decisão".

Para o teste com a amostra de 20 arquivos, captamos o assunto de 18 (90%), dentre os quais 18 estavam corretos (100%). Os arquivos com os quais tivemos problemas foram os mesmos da seção de datas, graças à desfiguração. Ela vai ser discutida na seção posterior.

### *Decisão*:
Normalmente, a decisão é introduzida sempre pela frase "Vistos, relatados e discutidos os autos". Usamos essa expressão regular para iniciar a captura. Ela estava presente em todas as decisões que captamos.

Além disso, normalmente termina com uma expressão específica. Normalmente, depois de relatar a decisão, o relator dá o prazo para apresentação de recursos (30 dias, em todos os arquivos que observamos). Por isso, usamos a expressão "30 dias" e suas variações como uma das "stopwords". Outra expressão muito usada é a data da sessão de julgamento, no formato por extenso. Usamos ela também. Além disso, em alguns casos, especialmente nos documentos mais recentes que analisamos, temos a expressão "Participaram desta Sessão", ou o fim da segunda página, como o fim da parte inicial dos autos.

Para o teste com a amostra de 20 arquivos, captamos a decisão em 18 (90%), dentre as quais todas continham as partes que julgamos relevantes (100%). Novamente, o problema foi na captação da decisão dos mesmos dois arquivos desfigurados. Apesar disso, houve, em muitos casos, a captura de mais texto do que necessário (uma frase ou alguns parágrafos a mais). Isso é um ponto de atenção, mas em todos os casos a decisão foi captada por completo (acreditamos que o limite de duas páginas na análise dos documentos foi importante nesse sentido).

## Possíveis perdas de dados (desconfiguração, documentos escaneados, etc.)
Como comentado na seção anterior, observamos, para dois documentos, problemas na colheita dos dados. Em ambos os casos, os documentos não tinham a primeira página na estrutura que esperávamos (Acusados, Assunto, Decisão). Muito pelo contrário. O que aconteceu foi que não conseguimos colher esses dados porque eles não existiam dentro dos arquivos. Apesar disso, os dados disponíveis foram completamente colhidos. Isso, acreditamos, é um indicativo de sucesso na acurácia do código (nenhuma informação falsa/incorreta colhida, nenhum "falso positivo").

No entanto, é importante saber, obviamente, que o código em questão não será capaz de colher a totalidade das informações dentro da pasta de mais de 900 arquivos baixados do site da CVM, mas acreditamos que boa parte estará lá.

Outro problema que certamente vamos enfrentar: uma pequena parte (só nos deparamos com um até o momento) dos processos são imagens escaneadas dos autos dos processos. Nesse caso, o código não vai funcionar e as informações terão de ser registradas manualmente.


----------

Vamos importar os pacotes a serem usados:

In [3]:
import os
import fitz
import pandas as pd
import re
import random
import shutil
import pdfplumber
from PyPDF2 import PdfReader
import numpy as np
import time

Vamos definir o caminho para a pasta que contém os arquivos:

In [2]:
# Caminho para a pasta principal com subpastas de anos
pasta_principal = "T:/T0mas/Faculdade/Bolsa PUB/Data"  # Altere para o caminho da sua pasta principal

Vamos ver com quantos arquivos estamos trabalhando.

In [4]:
# Dicionário para armazenar a contagem de arquivos por ano
contagem_arquivos = {}

# Contador total
total_arquivos = 0

# Loop pelas subpastas de anos (2000 a 2023)
for ano in range(2000, 2024):
    subpasta = os.path.join(pasta_principal, str(ano))  # Caminho para a subpasta do ano
    
    if os.path.exists(subpasta):  # Verifica se a subpasta existe
        # Contar arquivos PDF na subpasta
        arquivos_pdf = [arquivo for arquivo in os.listdir(subpasta) if arquivo.endswith('.pdf')]
        quantidade = len(arquivos_pdf)
        contagem_arquivos[ano] = quantidade
        total_arquivos += quantidade  # Soma ao total
    else:
        contagem_arquivos[ano] = 0  # Se a subpasta não existir, define contagem como 0

# Exibir os resultados
for ano, quantidade in contagem_arquivos.items():
    print(f"Ano {ano}: {quantidade} arquivos PDF")

print(f"\nNúmero total de arquivos PDF entre 2000 e 2023: {total_arquivos}")

Ano 2000: 19 arquivos PDF
Ano 2001: 36 arquivos PDF
Ano 2002: 30 arquivos PDF
Ano 2003: 41 arquivos PDF
Ano 2004: 62 arquivos PDF
Ano 2005: 80 arquivos PDF
Ano 2006: 110 arquivos PDF
Ano 2007: 58 arquivos PDF
Ano 2008: 35 arquivos PDF
Ano 2009: 48 arquivos PDF
Ano 2010: 45 arquivos PDF
Ano 2011: 24 arquivos PDF
Ano 2012: 25 arquivos PDF
Ano 2013: 57 arquivos PDF
Ano 2014: 41 arquivos PDF
Ano 2015: 55 arquivos PDF
Ano 2016: 62 arquivos PDF
Ano 2017: 50 arquivos PDF
Ano 2018: 98 arquivos PDF
Ano 2019: 99 arquivos PDF
Ano 2020: 66 arquivos PDF
Ano 2021: 23 arquivos PDF
Ano 2022: 50 arquivos PDF
Ano 2023: 57 arquivos PDF

Número total de arquivos PDF entre 2000 e 2023: 1271


Primeiro, definimos a função que vamos usar. Ela é a mesma do outro código.

In [5]:
# Função para extrair as informações de cada PDF
def extrair_informacoes_pdf(caminho_pdf):
    nome_arquivo = os.path.basename(caminho_pdf)
    pattern_processo_nome_arquivo = r"(?:\d{5}-\d{6}-\d{4}-\d{2}|IA-\d{4}-\d{2}|RJ-\d{4}-\d{5}|SP-\d{4}-\d{5})"
    match_processo_nome_arquivo = re.search(pattern_processo_nome_arquivo, nome_arquivo)
    numero_processo = match_processo_nome_arquivo.group(0) if match_processo_nome_arquivo else np.nan

    with pdfplumber.open(caminho_pdf) as pdf:
        text = ""
        for i in range(min(2, len(pdf.pages))):
            page = pdf.pages[i]
            text += page.extract_text()

    text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
    text = re.sub(r"https:\/\/.*\n", "", text)

    pattern_data_julgamento = r"(Data do julgamento:\s*(\d{2}/\d{2}/\d{4})|(?:Rio de Janeiro|São Paulo|Brasília),\s*(\d{2} de (?:janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de 20\d{2}))"
    pattern_acusados = r"(?:Acusados?\s*:|Interessados?\s*:|Indiciados?\s*:|ACUSADOS?\s*:|INTERESSADOS?\s*:|INDICIADOS?\s*:)\s*(.+?)(?=\s*(Assunto|Ementa|Relator|ASSUNTO|EMENTA|RELATOR))"
    pattern_assunto = r"(?:Assunto\s*:|Ementa\s*:|ASSUNTO\s*:|EMENTA\s*:)\s*(.+?)(?=\s*(Decisão|DECISÃO))" 
    pattern_decisao = r"(Vistos, relatados e discutidos os autos\s*(.+?))(?=\s*(?:30 dias|30 (trinta) dias|trinta dias|Participaram desta Sessão|(Rio de Janeiro|São Paulo|Brasília), \d{2} de (janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de 20\d{2}))"

    match_data_julgamento = re.search(pattern_data_julgamento, text)
    match_acusados = re.search(pattern_acusados, text, re.DOTALL)
    match_assunto = re.search(pattern_assunto, text, re.DOTALL)
    match_decisao = re.search(pattern_decisao, text, re.DOTALL)

    data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
    acusados = match_acusados.group(1).strip() if match_acusados else np.nan
    assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
    decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

    return numero_processo, data_julgamento, acusados, assunto, decisao

Agora, vamos fazer o loop que vai abrir as pastas de 2000 a 2024 e rodar a função para cada um dos arquivos dentro delas. Depois, ele vai adicionar tudo em um dataframe do pandas, que vamos exportar posteriormente.

In [6]:
# Lista para acumular os dados
dados = []

# Iniciar o timer total
start_time = time.time()

# Loop pelas subpastas de anos (2000 a 2023)
for ano in range(2000, 2024):
    subpasta = os.path.join(pasta_principal, str(ano))  # Caminho para a subpasta do ano
    
    if os.path.exists(subpasta):  # Verifica se a subpasta existe
        for arquivo in os.listdir(subpasta):  # Lista todos os arquivos na subpasta
            if arquivo.endswith('.pdf'):
                caminho_pdf = os.path.join(subpasta, arquivo)

                # Timer para cada iteração
                iter_start = time.time()

                try:
                    # Extrair informações do PDF
                    numero_processo, data_julgamento, acusados, assunto, decisao = extrair_informacoes_pdf(caminho_pdf)

                    # Criar lista de acusados
                    if isinstance(acusados, str):
                        pattern_nome_acusados = r"[^,\n]+(?:,[^,\n]+)*"  # Captura texto até o próximo '\n' ou vírgula interna
                        acusados_lista = [match.group(0).strip() for match in re.finditer(pattern_nome_acusados, acusados)]
                    else:
                        acusados_lista = []

                    # Adicionar dados à lista
                    dados.append({
                        "ID": numero_processo,
                        "Data": data_julgamento,
                        "Acusados": acusados_lista,
                        "Assunto": assunto,
                        "Decisão": decisao
                    })

                except Exception as e:
                    print(f"Erro ao processar {arquivo} na pasta {subpasta}: {e}")
                    continue

                # Timer de cada iteração
                iter_end = time.time()
                iter_time = iter_end - iter_start
                print(f"Tempo para processar {arquivo} da pasta {subpasta}: {iter_time:.2f} segundos")

# Criar o DataFrame com todos os dados
df = pd.DataFrame(dados)

# Calcular o tempo total
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo total de execução: {execution_time:.2f} segundos")

# Exibir o DataFrame com todos os resultados
df

Tempo para processar IA-1995-21.pdf da pasta T:/T0mas/Faculdade/Bolsa PUB/Data\2000: 0.53 segundos
Tempo para processar IA-1996-10.pdf da pasta T:/T0mas/Faculdade/Bolsa PUB/Data\2000: 0.65 segundos
Tempo para processar IA-1997-12.pdf da pasta T:/T0mas/Faculdade/Bolsa PUB/Data\2000: 0.97 segundos
Tempo para processar IA-1998-01.pdf da pasta T:/T0mas/Faculdade/Bolsa PUB/Data\2000: 0.73 segundos
Tempo para processar IA-1998-05_B.pdf da pasta T:/T0mas/Faculdade/Bolsa PUB/Data\2000: 0.54 segundos
Tempo para processar IA-1998-13.pdf da pasta T:/T0mas/Faculdade/Bolsa PUB/Data\2000: 0.73 segundos
Tempo para processar IA-1998-17.pdf da pasta T:/T0mas/Faculdade/Bolsa PUB/Data\2000: 0.66 segundos
Tempo para processar IA-1998-33.pdf da pasta T:/T0mas/Faculdade/Bolsa PUB/Data\2000: 0.67 segundos
Tempo para processar IA-1998-35.pdf da pasta T:/T0mas/Faculdade/Bolsa PUB/Data\2000: 0.79 segundos
Tempo para processar IA-1998-36_A.pdf da pasta T:/T0mas/Faculdade/Bolsa PUB/Data\2000: 0.77 segundos
Tempo 

,ID,Data,Acusados,Assunto,Decisão
0,IA-1995-21,"Rio de Janeiro, 08 de junho de 2000","[Alfredo Egydio Setúbal, Banco do Brasil S/A, ...",Práticas não-equitativas em operações day-trad...,"Vistos, relatados e discutidos os autos, o Col..."
1,IA-1996-10,"Rio de Janeiro, 09 de novembro de 2000","[Alfredo Egydio Setúbal, Banco Itaú S/A., Edua...",Práticas não-equitativas no mercado de valores...,"Vistos, relatados e discutidos os autos, o Col..."
2,IA-1997-12,NaN,"[Antônio Bosco da Costa, Antonio Carlos Martin...",Emissão pública de debêntures realizada pela E...,NaN
3,IA-1998-01,"Rio de Janeiro, 17 de agosto de 2000",[Bittencourt S/A CTVC Carlos Guilherme Frederi...,"Irregularidades na recepção, preenchimento, di...","Vistos, relatados e discutidos os autos, o Col..."
4,IA-1998-05,"Rio de Janeiro, 16 de novembro de 2000","[DC Corretora de Câmbio, Títulos e Valores Mob...",Operações fraudulentas no mercado futuro de ín...,"Vistos, relatados e discutidos os autos, o Col..."
...,...,...,...,...,...
1266,19957-011361-2018-48,Data do julgamento: 27/06/2023,"[YDUQS Participações S.A., Marcos de Oliveira ...",Apurar eventual responsabilidade de companhia ...,"Vistos, relatados e discutidos os autos, o Col..."
1267,19957-011669-2017-11,Data do julgamento: 19/09/2023,"[Luis Fernando Costa Estima, Estimapar Investi...",Apuração de eventuais irregularidades relacion...,"Vistos, relatados e discutidos os autos, o Col..."
1268,19957-012126-2022-70,Data do julgamento: 06/06/2023,"[Ingomar Mueller, Bruno Lippel, Multiplus Asse...",Apurar supostas práticas de administração irre...,"Vistos, relatados e discutidos os autos, o Col..."
1269,19957-012414-2022-24,Data do julgamento: 11/07/2023,"[José Almiro Bihl, Dirce Simioni Bihl, Aline C...",Apurar eventual responsabilidade de administra...,"Vistos, relatados e discutidos os autos, o Col..."


In [7]:
nan_count = df.isna().sum()
print(nan_count)

ID            0
Data        201
Acusados      0
Assunto      98
Decisão     220
dtype: int64


In [9]:
empty_lists_count = df['Acusados'].apply(lambda x: x == []).sum()
print(empty_lists_count)

38


In [10]:
rows_with_nan = df.isna().any(axis=1).sum()
print(rows_with_nan)

282


In [8]:
df.to_csv('T:/T0mas/Faculdade/Bolsa PUB/Github CVM/corporate-fraud-in-brazil/Bases de dados/Base de Dados Final.csv', index=False)